In [1]:
import time, random
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Input, InputLayer, Conv1D, MaxPooling1D,
                                     BatchNormalization, LeakyReLU, Flatten,
                                     LSTM, GRU, Dense, Dropout)
from tensorflow.keras.utils import to_categorical
import scipy.sparse as sp
from spektral.utils.sparse import sp_matrix_to_sp_tensor
from spektral.layers import ChebConv, GlobalAvgPool

# Reproducibility -------------------------------------------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Paths -----------------------------------------------------------------------
DATA_PATH = Path("emotions.csv")  # Change if needed
assert DATA_PATH.exists(), f"Could not find {DATA_PATH}."

# %% Load data ----------------------------------------------------------------

df = pd.read_csv(DATA_PATH)
print("Shape:", df.shape)


Shape: (2132, 2549)


In [2]:
# Heuristic label‑column detection
LABEL_COL = next((c for c in ["label", "emotion", "target", "class"] if c in df.columns), None)
if LABEL_COL is None:
    raise ValueError("Label column not detected - set LABEL_COL manually.")

X = df.drop(columns=LABEL_COL).astype("float32").values
labels = df[LABEL_COL].values

le = LabelEncoder()
y_int = le.fit_transform(labels)
num_classes = len(le.classes_)
print("Classes:", le.classes_)


Classes: ['NEGATIVE' 'NEUTRAL' 'POSITIVE']


In [3]:
y_cat = to_categorical(y_int, num_classes)

# Split -----------------------------------------------------------------------
X_train, X_test, y_train_int, y_test_int, y_train_cat, y_test_cat = train_test_split(
    X, y_int, y_cat, test_size=0.2, stratify=y_int, random_state=SEED)

# Scale (classical models + DL) ----------------------------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# Reshape for sequence models -------------------------------------------------
n_features = X_train_scaled.shape[1]  # sequence length
X_train_seq = X_train_scaled.reshape((-1, n_features, 1))
X_test_seq  = X_test_scaled.reshape((-1, n_features, 1))


In [4]:
K_NEIGH = 8  # number of neighbours each node connects to
A_dense = np.zeros((n_features, n_features), dtype="float32")
for i in range(n_features):
    for j in range(1, K_NEIGH // 2 + 1):
        A_dense[i, (i + j) % n_features] = 1
        A_dense[i, (i - j) % n_features] = 1
A_dense += A_dense.T + np.eye(n_features, dtype="float32")

A_sparse_sp = sp.csr_matrix(A_dense)                    # SciPy CSR
A_sparse_tf = sp_matrix_to_sp_tensor(A_sparse_sp)       # TF SparseTensor

In [5]:
def compile_and_train(model, inputs, targets, val_data, epochs=30, batch_size=64, lr=1e-3):
    # weight_decay through kernel_regularizer if needed; quick Adam setup
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(opt, loss="categorical_crossentropy", metrics=["accuracy"])
    t0 = time.time()
    model.fit(inputs, targets, validation_data=val_data,
              epochs=epochs, batch_size=batch_size, verbose=1)
    return time.time() - t0


def report(model, inputs, y_true_int, name="model"):
    y_pred_prob = model.predict(inputs, verbose=0)
    y_pred_int  = np.argmax(y_pred_prob, axis=1)
    acc = accuracy_score(y_true_int, y_pred_int)
    print(f"\n{name} – accuracy: {acc:.4f}\n")
    print(classification_report(y_true_int, y_pred_int, target_names=le.classes_))
    return acc

In [6]:
# Model builders --------------------------------------------------------------
def build_cnn(inp, classes):
    return Sequential([
        InputLayer(inp),
        Conv1D(64, 3, activation="relu"),
        MaxPooling1D(2),
        BatchNormalization(),
        Conv1D(128, 3, activation="relu"),
        MaxPooling1D(2),
        Flatten(),
        Dense(128, activation="relu"),
        Dropout(0.5),
        Dense(classes, activation="softmax"),
    ])


def build_lstm(inp, classes):
    return Sequential([
        InputLayer(inp),
        LSTM(128),
        Dropout(0.5),
        Dense(classes, activation="softmax"),
    ])


def build_gru(inp, classes):
    return Sequential([
        InputLayer(inp),
        GRU(128),
        Dropout(0.5),
        Dense(classes, activation="softmax"),
    ])


def build_cnn_lstm(inp, classes):
    return Sequential([
        InputLayer(inp),
        Conv1D(64, 3, activation="relu"),
        MaxPooling1D(2),
        BatchNormalization(),
        Conv1D(128, 3, activation="relu"),
        MaxPooling1D(2),
        LSTM(128),
        Dropout(0.5),
        Dense(classes, activation="softmax"),
    ])


def build_paper_cnn_lstm(inp, classes):
    m = Sequential()
    m.add(InputLayer(inp))
    for _ in range(10):
        m.add(Conv1D(64, 3, padding="same"))
        m.add(BatchNormalization())
        m.add(LeakyReLU(alpha=0.01))
        m.add(MaxPooling1D(2))
    m.add(Conv1D(64, 1))
    m.add(LSTM(128, return_sequences=True))
    m.add(LSTM(128, return_sequences=True))
    m.add(LSTM(128))
    m.add(Dense(100, activation="relu"))
    m.add(Dropout(0.5))
    m.add(Dense(classes, activation="softmax"))
    return m



class ChebConvNoMaskSparse(ChebConv):
    def __init__(self, *args, **kwargs):
        kwargs["sparse"] = True          # force sparse mode
        super().__init__(*args, **kwargs)

    def call(self, inputs):
        # Expect [X, A]; if mask sneaks in, drop it
        if isinstance(inputs, (list, tuple)) and len(inputs) == 3:
            inputs = inputs[:2]
        return super().call(inputs)

# ── DGCNN with *constant* adjacency ─────────────────────────────────────────
def build_dgcnn_constant(A_const_sparse, num_nodes, classes, channels=32):
    """
    A_const_sparse: tf.sparse.SparseTensor, shape = (num_nodes, num_nodes)
    """
    X_in = Input(shape=(num_nodes, 1), name="node_features")

    # Keras‑friendly way to inject a constant tensor into the graph:
    A_const = tf.keras.layers.Lambda(lambda _: A_const_sparse,
                                     name="const_adj")(X_in)

    x = ChebConvNoMaskSparse(channels, K=2, activation="relu")([X_in, A_const])
    x = Dropout(0.5)(x)
    x = GlobalAvgPool()(x)
    out = Dense(classes, activation="softmax")(x)
    return Model(inputs=X_in, outputs=out, name="DGCNN_constA")

In [7]:
results = []

In [8]:
# CNN -------------------------------------------------------------
name = "CNN"
print("\n==========", name ,"==========")
model = build_cnn((n_features, 1), num_classes)
model.summary()
rt = compile_and_train(model, X_train_seq, y_train_cat, (X_test_seq, y_test_cat), epochs=30)
acc = report(model, X_test_seq, y_test_int, name)
results.append((name, acc, rt))


========== CNN ==========


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 2546, 64)       │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 1273, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1273, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 1271, 128)      │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 635, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 81280)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    10,403,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,429,571 (39.79 MB)

 Trainable params: 10,429,443 (39.79 MB)

 Non-trainable params: 128 (512.00 B)

Epoch 1/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 79ms/step - accuracy: 0.6486 - loss: 8.9360 - val_accuracy: 0.8080 - val_loss: 0.5360
Epoch 2/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - accuracy: 0.8486 - loss: 0.5014 - val_accuracy: 0.8407 - val_loss: 0.5255
Epoch 3/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 73ms/step - accuracy: 0.9137 - loss: 0.2313 - val_accuracy: 0.8806 - val_loss: 0.4048
Epoch 4/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - accuracy: 0.9164 - loss: 0.2192 - val_accuracy: 0.9251 - val_loss: 0.3072
Epoch 5/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 73ms/step - accuracy: 0.9333 - loss: 0.1788 - val_accuracy: 0.9274 - val_loss: 0.2656
Epoch 6/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 79ms/step - accuracy: 0.9301 - loss: 0.1701 - val_accuracy: 0.9274 - val_loss: 0.2533
Epoch 7/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step - accuracy: 0.9282 - loss: 0.1558 - val_accuracy: 0.9368 - val_loss: 0.1854
Epoch 8/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 121ms/step - accuracy: 0.9513 - loss: 0.1301 - val_accuracy: 0.9508 - 

In [9]:
# LSTM -------------------------------------------------------------
name = "LSTM"
print("\n==========", name ,"==========")
model = build_lstm((n_features, 1), num_classes)
model.summary()
rt = compile_and_train(model, X_train_seq, y_train_cat, (X_test_seq, y_test_cat), epochs=30)
acc = report(model, X_test_seq, y_test_int, name)
results.append((name, acc, rt))


========== LSTM ==========


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128)            │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 66,947 (261.51 KB)

 Trainable params: 66,947 (261.51 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 112s 4s/step - accuracy: 0.4989 - loss: 1.0442 - val_accuracy: 0.5878 - val_loss: 0.7927
Epoch 2/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 145s 4s/step - accuracy: 0.6293 - loss: 0.8073 - val_accuracy: 0.5902 - val_loss: 0.8623
Epoch 3/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 116s 4s/step - accuracy: 0.6379 - loss: 0.7802 - val_accuracy: 0.8361 - val_loss: 0.6172
Epoch 4/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 121s 4s/step - accuracy: 0.7192 - loss: 0.6700 - val_accuracy: 0.8103 - val_loss: 0.7955
Epoch 5/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 123s 5s/step - accuracy: 0.7486 - loss: 0.8440 - val_accuracy: 0.7494 - val_loss: 0.6212
Epoch 6/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 126s 5s/step - accuracy: 0.7243 - loss: 0.6740 - val_accuracy: 0.8735 - val_loss: 0.5484
Epoch 7/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 126s 5s/step - accuracy: 0.7730 - loss: 0.6516 - val_accuracy: 0.7260 - val_loss: 0.7072
Epoch 8/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 141s 5s/step - accuracy: 0.7438 - loss: 0.7329 - val_accuracy: 0.7799 - v

In [10]:
# GRU -------------------------------------------------------------
name = "GRU"
print("\n==========", name ,"==========")
model = build_gru((n_features, 1), num_classes)
model.summary()
rt = compile_and_train(model, X_train_seq, y_train_cat, (X_test_seq, y_test_cat), epochs=30)
acc = report(model, X_test_seq, y_test_int, name)
results.append((name, acc, rt))


========== GRU ==========


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, 128)            │        50,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,691 (198.01 KB)

 Trainable params: 50,691 (198.01 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 190s 7s/step - accuracy: 0.4902 - loss: 1.0656 - val_accuracy: 0.7213 - val_loss: 0.9030
Epoch 2/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 202s 7s/step - accuracy: 0.6671 - loss: 0.8537 - val_accuracy: 0.7119 - val_loss: 0.7128
Epoch 3/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 203s 7s/step - accuracy: 0.6661 - loss: 0.7898 - val_accuracy: 0.8361 - val_loss: 0.7701
Epoch 4/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 189s 7s/step - accuracy: 0.7670 - loss: 0.7753 - val_accuracy: 0.7400 - val_loss: 0.5111
Epoch 5/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 192s 7s/step - accuracy: 0.7398 - loss: 0.5937 - val_accuracy: 0.8571 - val_loss: 0.4719
Epoch 6/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 190s 7s/step - accuracy: 0.7775 - loss: 0.5643 - val_accuracy: 0.8267 - val_loss: 0.5213
Epoch 7/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 204s 7s/step - accuracy: 0.7778 - loss: 0.5631 - val_accuracy: 0.8595 - val_loss: 0.4658
Epoch 8/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 200s 7s/step - accuracy: 0.8012 - loss: 0.5594 - val_accuracy: 0.8501 - v

In [11]:
# CNN-LSTM -------------------------------------------------------------
name = "CNN-LSTM"
print("\n==========", name ,"==========")
model = build_cnn_lstm((n_features, 1), num_classes)
model.summary()
rt = compile_and_train(model, X_train_seq, y_train_cat, (X_test_seq, y_test_cat), epochs=30)
acc = report(model, X_test_seq, y_test_int, name)
results.append((name, acc, rt))


========== CNN-LSTM ==========


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 2546, 64)       │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 1273, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1273, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 1271, 128)      │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 635, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 157,187 (614.01 KB)

 Trainable params: 157,059 (613.51 KB)

 Non-trainable params: 128 (512.00 B)

Epoch 1/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 26s 897ms/step - accuracy: 0.7419 - loss: 0.6768 - val_accuracy: 0.4286 - val_loss: 1.1050
Epoch 2/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 23s 862ms/step - accuracy: 0.8772 - loss: 0.3160 - val_accuracy: 0.4988 - val_loss: 1.1726
Epoch 3/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 41s 861ms/step - accuracy: 0.8780 - loss: 0.2946 - val_accuracy: 0.5340 - val_loss: 1.0465
Epoch 4/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 41s 870ms/step - accuracy: 0.8915 - loss: 0.2764 - val_accuracy: 0.5316 - val_loss: 0.9198
Epoch 5/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 24s 872ms/step - accuracy: 0.9071 - loss: 0.2574 - val_accuracy: 0.8478 - val_loss: 0.3516
Epoch 6/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 41s 859ms/step - accuracy: 0.9210 - loss: 0.2169 - val_accuracy: 0.5902 - val_loss: 0.7741
Epoch 7/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 41s 864ms/step - accuracy: 0.9371 - loss: 0.1707 - val_accuracy: 0.6885 - val_loss: 0.7230
Epoch 8/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 41s 859ms/step - accuracy: 0.9323 - loss: 0.1872 - val_accu

In [12]:
# CNN-LSTM (paper) -------------------------------------------------------------
name = "CNN-LSTM (paper)"
print("\n==========", name ,"==========")
model = build_paper_cnn_lstm((n_features, 1), num_classes)
model.summary()
rt = compile_and_train(model, X_train_seq, y_train_cat, (X_test_seq, y_test_cat), epochs=30)
acc = report(model, X_test_seq, y_test_int, name)
results.append((name, acc, rt))


========== CNN-LSTM (paper) ==========


c:\Users\Elite\anaconda3\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_4 (Conv1D)               │ (None, 2548, 64)       │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 2548, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 2548, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ (None, 1274, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 1274, 64)       │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 1274, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 1274, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ (None, 637, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_6 (Conv1D)               │ (None, 637, 64)        │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 637, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 637, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_6 (MaxPooling1D)  │ (None, 318, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_7 (Conv1D)               │ (None, 318, 64)        │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 318, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 318, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_7 (MaxPooling1D)  │ (None, 159, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_8 (Conv1D)               │ (None, 159, 64)        │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 159, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 159, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_8 (MaxPooling1D)  │ (None, 79, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_9 (Conv1D)               │ (None, 79, 64)         │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 79, 64)         │           256 │
│ (BatchNormalization)            │                        │             

 Total params: 493,331 (1.88 MB)

 Trainable params: 492,051 (1.88 MB)

 Non-trainable params: 1,280 (5.00 KB)

Epoch 1/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 16s 205ms/step - accuracy: 0.6361 - loss: 0.9182 - val_accuracy: 0.4005 - val_loss: 1.3694
Epoch 2/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 166ms/step - accuracy: 0.9113 - loss: 0.2932 - val_accuracy: 0.5457 - val_loss: 1.4656
Epoch 3/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 166ms/step - accuracy: 0.9356 - loss: 0.1952 - val_accuracy: 0.6042 - val_loss: 1.1122
Epoch 4/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 164ms/step - accuracy: 0.9574 - loss: 0.1500 - val_accuracy: 0.6838 - val_loss: 0.9896
Epoch 5/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 164ms/step - accuracy: 0.9614 - loss: 0.1229 - val_accuracy: 0.4098 - val_loss: 2.5998
Epoch 6/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 5s 163ms/step - accuracy: 0.9632 - loss: 0.1160 - val_accuracy: 0.9016 - val_loss: 0.3591
Epoch 7/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 163ms/step - accuracy: 0.9785 - loss: 0.0670 - val_accuracy: 0.5691 - val_loss: 2.6574
Epoch 8/30
27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 163ms/step - accuracy: 0.9814 - loss: 0.0604 - val_accuracy: 0

In [13]:
# SVM -------------------------------------------------------------
print("\n========== SVM (RBF) ==========")
svm = SVC(kernel="rbf", C=10, gamma="scale")
start = time.time()
svm.fit(X_train_scaled, y_train_int)
rt_svm = time.time() - start
svm_acc = accuracy_score(y_test_int, svm.predict(X_test_scaled))
print(f"SVM accuracy: {svm_acc:.4f}\n")
results.append(("SVM", svm_acc, rt_svm))


========== SVM (RBF) ==========
SVM accuracy: 0.9766



In [14]:
print("\n========== DGCNN (constant A) ==========")
dgcnn = build_dgcnn_constant(A_sparse_tf, n_features, num_classes)
dgcnn.summary()

# batch 64 now perfectly safe—only features in the batch dimension
rt_dg = compile_and_train(
    dgcnn,
    X_train_seq, y_train_cat,                  # X only!
    (X_test_seq, y_test_cat),
    epochs=20, batch_size=64, lr=1e-2
)

acc_dg = report(dgcnn, X_test_seq, y_test_int, "DGCNN")
results.append(("DGCNN", acc_dg, rt_dg))


========== DGCNN (constant A) ==========



Model: "DGCNN_constA"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ node_features       │ (None, 2548, 1)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ const_adj (Lambda)  │ (2548, 2548)      │          0 │ node_features[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cheb_conv_no_mask_… │ (None, 2548, 32)  │         96 │ node_features[0]… │
│ (ChebConvNoMaskSpa… │                   │            │ const_adj[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 2548, 32)  │          0 │ cheb_conv_no_mas… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_avg_pool     │ (None, 32)        │          0 │ dropout_5[0][0]   │
│ (GlobalAvgPool)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 3)         │         99 │ global_avg_pool[… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 195 (780.00 B)

 Trainable params: 195 (780.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.5483 - loss: 0.9687 - val_accuracy: 0.7143 - val_loss: 0.6792
Epoch 2/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.7225 - loss: 0.6697 - val_accuracy: 0.7166 - val_loss: 0.5767
Epoch 3/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.7468 - loss: 0.5725 - val_accuracy: 0.7471 - val_loss: 0.5124
Epoch 4/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.7723 - loss: 0.5111 - val_accuracy: 0.7916 - val_loss: 0.4523
Epoch 5/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.7965 - loss: 0.4614 - val_accuracy: 0.8080 - val_loss: 0.4192
Epoch 6/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.8221 - loss: 0.4262 - val_accuracy: 0.8056 - val_loss: 0.4021
Epoch 7/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.8352 - loss: 0.4034 - val_accuracy: 0.8197 - val_loss: 0.3826
Epoch 8/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.8482 - loss: 0.3833 - val_accuracy: 0.8314 - v

In [15]:
# Results summary ------------------------------------------------------------
summary = (pd.DataFrame(results, columns=["Model", "Accuracy", "Train_Time_sec"])
             .sort_values("Accuracy", ascending=False))
print("\n=== Summary ===")
print(summary.to_string(index=False))


=== Summary ===
           Model  Accuracy  Train_Time_sec
CNN-LSTM (paper)  0.995316      144.226198
             SVM  0.976581        1.255781
             CNN  0.974239      102.730754
        CNN-LSTM  0.964871     1092.247870
             GRU  0.908665     5904.657569
            LSTM  0.882904     3967.158851
           DGCNN  0.875878       12.015842
